In [1]:
from fastai.vision.all import *
from fastcore.parallel import *

import timm
import gc

In [2]:
path = Path('paddy-disease-classification')

train_path = path / 'train_images'
train_files = get_image_files(train_path)

test_path = path/'test_images'
test_files = get_image_files(test_path).sorted()

In [3]:
train_df = pd.read_csv(path / 'train.csv')
train_df.sample(10)

,image_id,label,variety,age
9086,101167.jpg,normal,Ponni,60
1172,105875.jpg,bacterial_panicle_blight,Ponni,80
3253,104667.jpg,brown_spot,ADT45,66
5258,100779.jpg,dead_heart,Ponni,80
1866,107858.jpg,blast,ADT45,70
6136,105505.jpg,hispa,ADT45,50
7607,103221.jpg,normal,ADT45,50
1115,110319.jpg,bacterial_panicle_blight,ADT45,77
8483,102862.jpg,normal,ADT45,65
3528,107438.jpg,brown_spot,ADT45,70


In [4]:
dblock = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    get_y=parent_label,
    splitter=RandomSplitter(0.2, seed=42),
    #item_tfms=Resize((640,480)),
    item_tfms=Resize(480),
    # size = 128 for faster training
    batch_tfms=aug_transforms(size=224, min_scale=0.75),
)

In [5]:
dls = dblock.dataloaders(train_path, bs=16)

In [6]:
arch = 'vit_large_patch16_224'
cbs = GradientAccumulation(64)

learn = vision_learner(dls, arch, metrics=error_rate, cbs=cbs).to_fp16()

In [7]:
learn.fine_tune(1, 0.01)

epoch,train_loss,valid_loss,error_rate,time
0,1.067428,0.647612,0.205190,10:18


epoch,train_loss,valid_loss,error_rate,time


RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 8.00 GiB total capacity; 6.85 GiB already allocated; 0 bytes free; 7.32 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [16]:
!ls /home/fmussari/.cache/torch/hub/checkpoints/

L_16-i21k-300ep-lr_0.001-aug_medium1-wd_0.1-do_0.1-sd_0.1--imagenet2012-steps_20k-lr_0.01-res_224.npz
convnext_small_22k_224.pth
convnext_tiny_22k_224.pth
resnet18-5c106cde.pth
resnet18-f37072fd.pth
resnet34-43635321.pth
resnet34-b627a593.pth
resnet50-0676ba61.pth
resnet50_a1_0-14fe96d1.pth
swinv2_base_patch4_window12_192_22k.pth
